# 1D Model with Beverton-Holt Stock-Recruitment and Bednarsek Mortality

This notebook demonstrates the use of the AcidityBedBHSurvivalModel, which combines:

-   Bednarsek et al. (2022) linear mortality equation for ocean acidification effects
-   Beverton-Holt density-dependent stock-recruitment relationship
-   Survival rate modulation of recruitment

The Beverton-Holt function provides density-dependent regulation where recruitment efficiency
depends on spawning stock biomass: f(SSB) = (b _ SSB) / (1 + b _ SSB)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

from seapopym.configuration.acidity import (
    ForcingParameter,
)
from seapopym.configuration.acidity_bed_bh import (
    AcidityBedBHConfiguration,
    FunctionalGroupParameter,
    FunctionalGroupUnit,
    FunctionalTypeParameter,
)
from seapopym.configuration.no_transport import (
    ForcingUnit,
    MigratoryTypeParameter,
)
from seapopym.model import AcidityBedBHSurvivalModel
from seapopym.standard.coordinate_authority import (
    create_latitude_coordinate,
    create_layer_coordinate,
    create_longitude_coordinate,
    create_time_coordinate,
)
from seapopym.standard.units import StandardUnitsLabels

## Generating data for the 1D simulation

We generate synthetic data for temperature, primary production, and acidity. The Bednarsek model uses
these environmental variables to compute mortality and survival rate, while the Beverton-Holt function
regulates recruitment based on spawning stock biomass.


In [ ]:
time_axis = create_time_coordinate(pd.date_range("2000-01-01", "2001-01-01", freq="D"))
n = int(time_axis.size)
t = np.linspace(0, 1, n)
temperature = np.ones_like(t) * 10
acidity = 2 + 0.5 * np.cos(2 * np.pi * t**2)

temperature = xr.DataArray(
    dims=["T", "Y", "X", "Z"],
    coords={
        "T": create_time_coordinate(pd.date_range("2000-01-01", "2001-01-01", freq="D")),
        "Y": create_latitude_coordinate([0]),
        "X": create_longitude_coordinate([0]),
        "Z": create_layer_coordinate([0]),
    },
    attrs={"units": StandardUnitsLabels.temperature},
    data=temperature[:, np.newaxis, np.newaxis, np.newaxis],
)

plt.figure(figsize=(9, 3))
temperature[:, 0, 0].cf.plot.line(x="T")
plt.title("Temperature")
plt.show()

acidity = xr.DataArray(
    dims=["T", "Y", "X", "Z"],
    coords={
        "T": create_time_coordinate(pd.date_range("2000-01-01", "2001-01-01", freq="D")),
        "Y": create_latitude_coordinate([0]),
        "X": create_longitude_coordinate([0]),
        "Z": create_layer_coordinate([0]),
    },
    attrs={"units": StandardUnitsLabels.acidity},
    data=acidity[:, np.newaxis, np.newaxis, np.newaxis],
)

plt.figure(figsize=(9, 3))
acidity[:, 0, 0].cf.plot.line(x="T")
plt.title("Aragonite Saturation State")
plt.show()

primary_production = xr.DataArray(
    dims=["T", "Y", "X"],
    coords={
        "T": create_time_coordinate(pd.date_range("2000-01-01", "2001-01-01", freq="D")),
        "Y": create_latitude_coordinate([0]),
        "X": create_longitude_coordinate([0]),
    },
    attrs={"units": "g/m2/day"},
    data=np.full((n, 1, 1), 1),
)

plt.figure(figsize=(9, 3))
primary_production.plot()
plt.title("Primary Production")
plt.show()

dataset = xr.Dataset({"temperature": temperature, "primary_production": primary_production, "acidity": acidity})

In [ ]:
biomass = xr.DataArray(
    dims=["functional_group", "Y", "X"],
    coords={
        "functional_group": [0],
        "Y": create_latitude_coordinate([0]),
        "X": create_longitude_coordinate([0]),
    },
    attrs={"units": "g/m2"},
    data=np.full((1, 1, 1), 0.02),
    name="initial_biomass",
)
biomass.functional_group.attrs = {
    "flag_values": "range(0, 1)",
    "flag_meanings": "Pteropod",
    "standard_name": "functional_group",
    "long_name": "functional group",
}

## Initialize the Beverton-Holt + Bednarsek model

We add the `density_dependance_parameter` (b = 0.2) to the functional type parameters.
This parameter controls the strength of density-dependent recruitment limitation in the
Beverton-Holt equation.

**Important**: We also need to provide initial biomass conditions to kickstart the Beverton-Holt
recruitment. Without spawning stock biomass (SSB=0), the BH coefficient would be 0 and no
recruitment would occur.


In [ ]:
f_groups = FunctionalGroupParameter(
    functional_group=[
        FunctionalGroupUnit(
            name="Pteropod",
            energy_transfert=1,
            migratory_type=MigratoryTypeParameter(day_layer=0, night_layer=0),
            functional_type=FunctionalTypeParameter(
                lambda_0=-19.4,
                gamma_lambda_temperature=11.5,
                gamma_lambda_acidity=-32.7,
                survival_rate_0=13.49,
                gamma_survival_rate_temperature=-2.475,
                gamma_survival_rate_acidity=10.10,
                tr_0=10.38,
                gamma_tr=-0.11,
                density_dependance_parameter=1,  # Beverton-Holt parameter
            ),
        )
    ]
)

p_param = ForcingParameter(
    temperature=ForcingUnit(forcing=dataset["temperature"]),
    primary_production=ForcingUnit(forcing=dataset["primary_production"]),
    acidity=ForcingUnit(forcing=dataset["acidity"]),
    initial_condition_biomass=ForcingUnit(forcing=biomass),
)

parameters = AcidityBedBHConfiguration(forcing=p_param, functional_group=f_groups)

## Run the Beverton-Holt + Bednarsek model

The AcidityBedBHSurvivalModel integrates:

1. Bednarsek linear mortality equation
2. Bednarsek survival rate applied to recruitment
3. Beverton-Holt density-dependent stock-recruitment

The biomass dynamics follow: biomass(t-1) → BH coefficient → recruitment → survival rate → biomass(t)


In [ ]:
with AcidityBedBHSurvivalModel.from_configuration(configuration=parameters) as beverton_holt_model:
    beverton_holt_model.run()
    display(beverton_holt_model.state)
    state = beverton_holt_model.state.copy()

## Plotting the results

### Biomass evolution with Beverton-Holt recruitment and Bednarsek mortality

The biomass shows the combined effects of:

-   Density-dependent recruitment limitation (Beverton-Holt)
-   Ocean acidification effects on mortality (Bednarsek)
-   Survival rate modulation of recruitment (Bednarsek)


In [ ]:
fig, axes = plt.subplots(5, 1, figsize=(10, 15))
state.sel(T=slice("2000-02-01", None)).biomass.squeeze().plot.line(x="T", hue="functional_group", ax=axes[0])
axes[0].set_title("Biomass (with Beverton-Holt recruitment)")

state.sel(T=slice("2000-02-01", None)).acidity.plot(ax=axes[1])
axes[1].set_title("Aragonite Saturation State")

state.sel(T=slice("2000-02-01", None)).temperature.plot(ax=axes[2])
axes[2].set_title("Temperature")

state.sel(T=slice("2000-02-01", None)).survival_rate.plot(ax=axes[3])
axes[3].set_title("Survival Rate (Bednarsek)")

state.sel(T=slice("2000-02-01", None)).mortality_field.plot(ax=axes[4])
axes[4].set_title("Mortality Field (Bednarsek)")

for ax in axes:
    ax.grid()
    ax.set_xlabel("")

plt.tight_layout()
plt.show()

## Comparison with standard Bednarsek model (Optional)

To compare the effects of Beverton-Holt density dependence, you can run the standard
AcidityBedModel side-by-side and observe the differences in biomass dynamics.

The Beverton-Holt model should show:

-   Reduced biomass growth at high densities (density-dependent limitation)
-   More stable equilibrium biomass levels
-   Different response to environmental forcing compared to the linear model


## Summary

This notebook demonstrated the AcidityBedBHSurvivalModel which integrates:

1. **Beverton-Holt stock-recruitment**: Provides density-dependent recruitment limitation
2. **Bednarsek mortality equation**: Accounts for ocean acidification and temperature effects on mortality
3. **Bednarsek survival rate**: Modulates recruitment based on environmental conditions

The density_dependance_parameter (b=0.2) controls how quickly recruitment saturates with increasing
spawning stock biomass. Higher values lead to faster saturation.

**Key parameters**:

-   `density_dependance_parameter = 0.2`: Moderate density dependence
-   At SSB = 1/0.2 = 5, the Beverton-Holt coefficient reaches 0.5 (inflection point)
-   Maximum recruitment efficiency approaches 1.0 at high spawning stock biomass
